In [10]:
import numpy as np
import pandas as pd
import numpy as np
import os
import time
from google.cloud import bigquery
import re

In [3]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados para subir\\Tabelas finais')


In [11]:
df = pd.read_csv('SIAPE_servidores_agrupamento_carreira.csv')
df

,ano,carreiras,total
0,2004,social,167438
1,2005,social,167511
2,2006,social,178166
3,2007,social,178301
4,2008,social,182809
...,...,...,...
93,2013,nao_classificado,42339
94,2014,nao_classificado,42277
95,2015,nao_classificado,41415
96,2016,nao_classificado,37701


In [13]:
df['carreiras'] = df['carreiras'].str.title()

In [14]:
df

,ano,carreiras,total
0,2004,Social,167438
1,2005,Social,167511
2,2006,Social,178166
3,2007,Social,178301
4,2008,Social,182809
...,...,...,...
93,2013,Nao_Classificado,42339
94,2014,Nao_Classificado,42277
95,2015,Nao_Classificado,41415
96,2016,Nao_Classificado,37701


In [16]:
def transformar(nome):
    nome = re.sub(r"\bEconomia_Financas_Gestao\b", "Economia, Finanças e Gestão", nome)
    nome = re.sub(r"\bInfraestrutura_Desenvolvimento_Pesquisa\b", "Infraestrutura, Desenvolvimento e Pesquisa", nome)
    nome = re.sub(r"\bJuridico_Policial\b", "Jurídico e Policial", nome)
    nome = re.sub(r"\bApoio_Tecnico_Administrativo\b", "Apoio Técnico e Administrativo", nome)
    nome = re.sub(r"\bRegulacao_Auditoria_Fiscalizacao\b", "Regulação, Auditoria e Fiscalização", nome)
    nome = re.sub(r"\bNao_Classificado\b", "Não Classificado", nome)
    
    return nome

In [17]:
df['carreiras'] = df['carreiras'].apply(transformar)

In [19]:
df['carreiras'].unique()

array(['Social', 'Economia, Finanças e Gestão',
       'Infraestrutura, Desenvolvimento e Pesquisa',
       'Jurídico e Policial', 'Apoio Técnico e Administrativo',
       'Regulação, Auditoria e Fiscalização', 'Não Classificado'],
      dtype=object)

In [18]:
df

,ano,carreiras,total
0,2004,Social,167438
1,2005,Social,167511
2,2006,Social,178166
3,2007,Social,178301
4,2008,Social,182809
...,...,...,...
93,2013,Não Classificado,42339
94,2014,Não Classificado,42277
95,2015,Não Classificado,41415
96,2016,Não Classificado,37701


In [20]:
df = df.rename(columns={'carreiras':'carreiras_area', 'total':'valor'}) #Padronizando
df 

,ano,carreiras_area,valor
0,2004,Social,167438
1,2005,Social,167511
2,2006,Social,178166
3,2007,Social,178301
4,2008,Social,182809
...,...,...,...
93,2013,Não Classificado,42339
94,2014,Não Classificado,42277
95,2015,Não Classificado,41415
96,2016,Não Classificado,37701


Subindo para o GBQ

In [21]:
client = bigquery.Client()
dataset_ref = client.dataset('estrutura_organizacional_carreiras')

c:\Users\ana.sales_republica\Anaconda\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [22]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('carreiras_area','STRING',description='Grupo de carreiras da observação'),
        bigquery.SchemaField('valor','INTEGER',description='Quantidade total da observação')]


In [23]:
table_ref = dataset_ref.table('SIAPE_servidores_agrupamento_carreira') 
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 


LoadJob<project=repositoriodedadosgpsp, location=US, id=96e3fd1f-0840-4503-8097-00dd1a6ee3a3>